# Metadata

```
Course:  DS 5001
Module:  03 Lab
Topic:   Simple Language Model
Author:  R.C. Alvarado
Date:    31 January 2023
```

**Purpose**: Demonstrates use of simple langauge model based on Berkeley restaurant data (from Jurafsky and Martin).

In [160]:
import pandas as pd
import numpy as np

We convert these data into Pandas dataframes and use them to predict and generate sentences.

<img src="lm-data.png">

In [48]:
data1 = dict(
    w0 = "i, want, to, eat, chinese, food, lunch, spend".split(", "),
    n = [2533, 927, 2417, 746, 158, 1093, 341, 278]
)

In [54]:
df1 = pd.DataFrame(data1).set_index(['w0'])
df1.n = df1.n.astype('int')

In [151]:
df1['p'] = df1.n / df1.n.sum()

In [153]:
df1

,n,p
w0,,
i,2533,0.298246
want,927,0.109149
to,2417,0.284587
eat,746,0.087837
chinese,158,0.018604
food,1093,0.128694
lunch,341,0.040151
spend,278,0.032733


In [161]:
data2 = [row.split(", ") for row in """
i, i, 5
i, want, 837
i, eat, 9
i, spend, 2
want, i, 2
want, to, 608
want, eat, 1
want, chinese, 6
want, food, 6
want, lunch, 5
want, spend, 1
to, i, 2
to, to, 4
to, eat, 686
to, chinese, 2
to, lunch, 6
to, spend, 211
eat, to, 2
eat, chinese, 16
eat, food, 2
eat, lunch, 42
chinese, i, 1
chinese, food, 82
chinese, lunch, 1
food, i, 15
food, to, 15
food, chinese, 1
food, food, 4
lunch, i, 2
lunch, food, 1
spend, i, 1
spend, to, 1
""".split("\n")[1:-1]]

In [162]:
df2 = pd.DataFrame(data2, columns = ['w0', 'w1', 'n']).set_index(['w0','w1'])#.unstack(fill_value=0)

In [163]:
df2.n = df2.n.astype('int')

In [165]:
df2

n
w0      w1          
i       i          5
        want     837
        eat        9
        spend      2
want    i          2
        to       608
        eat        1
        chinese    6
        food       6
        lunch      5
        spend      1
to      i          2
        to         4
        eat      686
        chinese    2
        lunch      6
        spend    211
eat     to         2
        chinese   16
        food       2
        lunch     42
chinese i          1
        food      82
        lunch      1
food    i         15
        to        15
        chinese    1
        food       4
lunch   i          2
        food       1
spend   i          1
        to         1

In [164]:
df3 = (df2.n / df1.n).unstack(fill_value=0)

In [76]:
df3.style.background_gradient(cmap='YlGnBu', axis=None)

w1,chinese,eat,food,i,lunch,spend,to,want
w0,,,,,,,,
chinese,0.000000,0.000000,0.518987,0.006329,0.006329,0.000000,0.000000,0.000000
eat,0.021448,0.000000,0.002681,0.000000,0.056300,0.000000,0.002681,0.000000
food,0.000915,0.000000,0.003660,0.013724,0.000000,0.000000,0.013724,0.000000
i,0.000000,0.003553,0.000000,0.001974,0.000000,0.000790,0.000000,0.330438
lunch,0.000000,0.000000,0.002933,0.005865,0.000000,0.000000,0.000000,0.000000
spend,0.000000,0.000000,0.000000,0.003597,0.000000,0.000000,0.003597,0.000000
to,0.000827,0.283823,0.000000,0.000827,0.002482,0.087298,0.001655,0.000000
want,0.006472,0.001079,0.006472,0.002157,0.005394,0.001079,0.655879,0.000000


In [159]:
N = 20
M = 10

# Generate N sentences
for i in range(N):
    print(str(i+1).zfill(2), end = ' ')
    w0 = df1.sample(weights='n').index[0]
    ll = np.log(df1.loc[w0].p)
    
    # Select M words for each sentence
    for j in range(M):
        print(w0, end=' ')
        w1 = df3.loc[w0].sample(weights=df3.loc[w0].values).index[0]
        ll += np.log(df3.loc[w0, w1])
        w0 = w1
    
    # Add log likelihood
    print(round(ll))

01 to eat chinese food to spend i want to eat -26
02 to eat lunch i want to spend i want food -31
03 eat lunch i want to eat chinese food to spend -30
04 i want to eat lunch food i want to eat -23
05 to chinese food i want to eat lunch food to -30
06 eat lunch i want to eat lunch food i want -28
07 want to eat chinese food i want to eat lunch -23
08 food i want to eat chinese food i want to -21
09 i want to spend to eat chinese food i want -22
10 to eat lunch i want to eat lunch i want -23
11 food i want to eat lunch food food i want -29
12 i want to spend to eat lunch food to eat -29
13 food food i want to spend to eat lunch food -36
14 to spend i want to eat food to eat lunch -32
15 i want to eat chinese food i want to eat -19
16 i want to spend i want to eat chinese food -22
17 spend i want to eat chinese food to eat lunch -31
18 to eat lunch i want to eat lunch i want -23
19 to eat lunch i want to eat lunch i want -23
20 food i want to spend to eat lunch food to -31


In [135]:
df1.sample(weights='n').index[0]

'to'

In [139]:
df3.loc['i','want']

0.3304382155546782